In [2]:
# Importing necessary libraries for Parquet
import pyarrow.csv as csv
import pyarrow.parquet as pq
import glob
import os
import pyarrow as pa
from pyarrow import parquet

In [35]:
# Providing folder path
folder_path = "/Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/"

# Partquet file name
parquet_file = "combined_place_data.parquet"

# Getting csv_file names
csv_files = glob.glob(os.path.join(folder_path, '**/*.csv'), recursive=True)

# Create Schema
schema = pa.schema([
    pa.field("timestamp", pa.string()),
    pa.field("user", pa.string()),
    pa.field("coordinate", pa.string()),
    pa.field("pixel_color", pa.string())
])

# Initializing writer
writer = parquet.ParquetWriter(parquet_file, schema, compression='snappy')

# Iterating through csv files
for file in sorted(csv_files):
    
    print("reading", file)
    table = csv.read_csv(file)
    # Casting table to schema
    table = table.cast(schema)

    print("writing", file)
    writer.write_table(table)
    
if writer:
    writer.close()
    
print(f"Combined CSVs to {parquet_file}")

reading /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000000.csv
writing /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000000.csv
reading /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000001.csv
writing /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000001.csv
reading /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000002.csv
writing /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000002.csv
reading /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000003.csv
writing /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000003.csv
reading /Users/ben/Documents/GitHub/CloudComputing/RedditPlaceData/2023_place_canvas_history-000000000004.csv
writing /U

In [81]:
import polars as pl
import pyarrow.parquet as pq
from datetime import datetime

In [2]:
test_file = pq.ParquetFile("/Users/ben/Documents/GitHub/CloudComputing/combined_place_data.parquet")

In [3]:
print(f"Number of entries (rows) in the Parquet file: {test_file.metadata.num_rows:,d}")

Number of entries (rows) in the Parquet file: 129,788,513


In [4]:
print("read parquet")
# Read the Parquet file
df = pl.scan_parquet('combined_place_data.parquet')
df = df.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S%.f %Z').cast(pl.Datetime, strict=False))

read parquet


In [9]:
value_counts = df.select("user").group_by("user").agg(pl.col('user').count().alias('count')).sort('count', descending=True).collect(streaming = True)

In [57]:
value_counts.select("count").quantile(quantile=.90).collect(streaming=True)

count
f64
37.0


In [41]:
value_counts = value_counts.with_columns(pl.col("count").cast(int))

In [1]:
pbs_idx = value_counts.filter(pl.col("count") >= 37)

NameError: name 'value_counts' is not defined

In [91]:
df.filter(pl.col('user').is_in(pbs_idx.select("user").collect(streaming=True))).collect(streaming=True)

timestamp,user,coordinate,pixel_color
str,str,str,str
"""2023-07-20 13:…","""no+8HEIDjbdx7/…","""-199,-235""","""#FFFFFF"""
"""2023-07-20 13:…","""uqi5XwkBePwcPK…","""-42,-218""","""#FFFFFF"""
"""2023-07-20 13:…","""2bmivBNj8NYvnp…","""182,164""","""#FF4500"""
"""2023-07-20 13:…","""iyPavVpo8ojDYs…","""-113,-1""","""#FFFFFF"""
"""2023-07-20 13:…","""a6Q+OsCSRDcPxh…","""-64,-34""","""#3690EA"""
"""2023-07-20 13:…","""AS0KN9rxoynWuN…","""-267,-142""","""#FFFFFF"""
"""2023-07-20 13:…","""vFCy3asEWbBER9…","""-43,74""","""#000000"""
"""2023-07-20 13:…","""rkTj/gktE+fk1u…","""-279,242""","""#000000"""
"""2023-07-20 13:…","""sxYEVPfaohJYIY…","""60,-65""","""#000000"""


In [55]:
pbs_idx.collect()

user,count
str,u32
"""kN91vAk/cntIE4…",7046
"""oCgWHk6Cfsa52v…",6285
"""/YNNvIErSA5atG…",5331
"""ay5U7RmL4QAj/p…",4390
"""bRwFe1qJhazPc7…",4184
"""tFRL97+0mluUiL…",4046
"""gAjvJgodeKO2IW…",3712
"""/XIhfEKO27ngFq…",3413
"""nNbAayT6hRtQHF…",3344
